In [1]:
import pandas as pd 
train = pd.read_csv(open(r"C:\Users\qian_wang\Desktop\疫情\NCPPolicies_train_20200301.csv",encoding = "utf-8"),delimiter = "\t")
test = pd.read_csv(open(r"C:\Users\qian_wang\Desktop\疫情\NCPPolicies_test.csv",encoding = "utf-8"),delimiter = "\t")
doc = pd.read_csv(open(r"C:\Users\qian_wang\Desktop\疫情\NCPPolicies_context_20200301.csv",encoding = "utf-8"),delimiter = "\n")
doc_pro = pd.DataFrame()
doc_pro["docid"] = [text[0].split("\t")[0] for text in doc.values]
doc_pro["text"] = [text[0].split("\t")[1] for text in doc.values]
#doc_pro.to_csv(r"C:\Users\qian_wang\Desktop\疫情\doc_new.csv",index = False)
#train.to_csv(r"C:\Users\qian_wang\Desktop\疫情\train_new.csv",index = False)
#test.to_csv(r"C:\Users\qian_wang\Desktop\疫情\test_new.csv",index = False)
#doc_pro去重
#doc_pro.drop_duplicates(subset = ["text"],keep='first',inplace = True)
doc_pro.shape

(8932, 2)

In [2]:
train_doc = pd.merge(train,doc_pro,how = "left",on = "docid")
#train_doc.to_csv(r"C:\Users\qian_wang\Desktop\疫情\train_doc.csv",index = False)

In [3]:
import jieba.posseg as pseg
from gensim import corpora
from gensim.summarization import bm25

stop_words = []

for i in open(r"C:\Users\qian_wang\Desktop\疫情\stop_words.txt",encoding = "utf-8").readlines():
    stop_words.append(i.strip())
stop_flag = ['x', 'c', 'u','d', 'p', 't', 'uj', 'm', 'f', 'r']
def tokenization(texts):
    result = []
    for word in texts["text"].values:
        words = pseg.cut(word)
        out = []
        for word, flag in words:
            
            if flag not in stop_flag and word not in stop_words:
            #if flag not in stop_flag:
                out.append(word)
        result.append(out)
    return result
#构建语料库
corpus = []
corpus = tokenization(doc_pro)
#构建bm25模型
bm25Model = bm25.BM25(corpus)
average_idf = sum(map(lambda k: float(bm25Model.idf[k]),bm25Model.idf.keys())) / len(bm25Model.idf.keys())
#判断相关性
def top_correlation_index(query,bm25Model,average_idf,stop_flag):
    
    #query_word = [" ".join(jieba.cut(query)).split()]
    words = pseg.cut(query)
    out = []
    for word, flag in words:
        if flag not in stop_flag:
                out.append(word)
            
    scores = bm25Model.get_scores(out,average_idf)
    
    return scores.index(max(scores))

#构建docid-index

docid_index = {}
for i,docid in enumerate(doc_pro["docid"].values):
    docid_index[i] = docid
"""
text_doc_id = []
for query in test["question"]:
    
    text_doc_id.append(docid_index[top_correlation_index(query,bm25Model,average_idf,stop_flag)])"""   

D:\wenjiananzhaung\Anaconda\lib\site-packages\gensim\utils.py:1209: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Building prefix dict from the default dictionary ...
Dumping model to file cache C:\Users\QIAN_W~1\AppData\Local\Temp\jieba.cache
Loading model cost 1.182 seconds.
Prefix dict has been built succesfully.


'\ntext_doc_id = []\nfor query in test["question"]:\n    \n    text_doc_id.append(docid_index[top_correlation_index(query,bm25Model,average_idf,stop_flag)])'

In [20]:
#生成top_N
def topN_doc(query,bm25Model,average_idf,stop_flag,N):
    words = pseg.cut(query)
    out = []
    for word, flag in words:
        if flag not in stop_flag:
                out.append(word)
            
    scores = bm25Model.get_scores(out,average_idf)
    res = {}
    for i,v in enumerate(scores):
        res[i] = v
        
    return sorted(res.items(),key = lambda x:x[1],reverse = True)[:N]
def train_y_docid(train_doc,bm25Model,average_idf,stop_flag,N):
    id_ = []
    docid_ = []
    question_ = []
    answer_ = []
    y = []
    for i in range(train_doc.shape[0]):
        id_.extend([train_doc.iloc[i,0]]*N)
        question_.extend([train_doc.iloc[i,2]]*N)
    
        res = topN_doc(train_doc.iloc[i,2],bm25Model,average_idf,stop_flag,N)
        temp = [docid_index[i[0]] for i in res]
        if train_doc.iloc[i,1] in temp:
            for j in temp:
                if train_doc.iloc[i,1]==j:
                    y.append(1)
                    answer_.append(train_doc.iloc[i,3])
               
                else:
                    
                    y.append(0)
                    answer_.append(train_doc.iloc[i,3])
                docid_.append(j)
        else:
            docid_.append(train_doc.iloc[i,1])
            docid_.extend(temp[:(N-1)])
            answer_.append(train_doc.iloc[i,3])
            answer_.extend([train_doc.iloc[i,3]]*(N-1))
            y.append(1)
            y.extend([0]*(N-1))
    return pd.DataFrame({"id":id_,"docid":docid_,"question":question_,"answer":answer_,"y":y})
train_doc_N3 = train_y_docid(train_doc,bm25Model,average_idf,stop_flag,10)

In [21]:
train_doc_N3_docid = pd.merge(train_doc_N3,doc_pro,how = "left",on = "docid")
train_doc_N3_docid.head()

,id,docid,question,answer,y,text
0,47a41a03966431739257ef215cdc1caa,015758c216923f89991ca61c67b29f70,工业和信息化部到哪家企业进行督导检查？,北京北铃专用汽车有限公司,1,工业和信息化部组织开展负压救护车重点生产企业督导检查 2020年2月4日，为做好新型冠状病毒...
1,47a41a03966431739257ef215cdc1caa,028b66062e28322ebf200b483ecc31ee,工业和信息化部到哪家企业进行督导检查？,北京北铃专用汽车有限公司,0,关于启动我市重污染天气橙色预警（Ⅱ级） 各县、区人民政府、城乡一体化示范区管委会，市重污染天...
2,47a41a03966431739257ef215cdc1caa,9377a8eccd1939bc94b709ee39c60587,工业和信息化部到哪家企业进行督导检查？,北京北铃专用汽车有限公司,0,关于启动我市重污染天气橙色预警（Ⅱ级） 各县、区人民政府，城乡一体化示范区管委会、市重污染天...
3,47a41a03966431739257ef215cdc1caa,7e4dd892d36f38eca074e21d7315e674,工业和信息化部到哪家企业进行督导检查？,北京北铃专用汽车有限公司,0,张洪谱带队开展岁末年初安全生产督导检查工作 为贯彻落实我市交通运输行业岁末年初安全生产各项措...
4,47a41a03966431739257ef215cdc1caa,4418cb19fa963f1a8977c6d3017ef55e,工业和信息化部到哪家企业进行督导检查？,北京北铃专用汽车有限公司,0,关于将我市重污染天气橙色预警（Ⅱ级） 各县（区）人民政府、城乡一体化示范区管委会、市重污染天...


In [22]:
train_doc_N3_docid.shape

(50000, 6)

In [23]:
count = 0
for row in train_doc_N3_docid.itertuples():
    if row.text.find(row.answer)!=-1:
        count+=1
count

9911

In [33]:
max_context_len = 2000

def get_data(ds):
    data = {
        "version": "v2.0",
        "data": [

        ]
    }
    
    answerable = 0
    unanswerable = 0
    parted = 0
    
    for row in tqdm(ds.itertuples(), total=len(ds)):
        question = re.sub(r'\s+', ' ', row.question).strip()
        answer = re.sub(r'\s+', ' ', row.answer).strip()
        context = re.sub(r'\s+', ' ', row.text).strip()
        
        finds = find_all(context, answer)
        
        if len(finds) == 0:
            if len(answer) > 20:
                cxt_len = max(len(answer) + max_context_len // 2, max_context_len)
                
                if len(context) > cxt_len:
                    start = max(0, random.randrange(0, len(context) - cxt_len))
                    context = context[start: start + cxt_len]
                
                qas = [{
                    'question': question,
                    'id': row.id + '_' + row.docid,
                    'answers': [],
                    'is_impossible': True,
                }]
                
                d = {
                    'title': row.docid,
                    'paragraphs': [{
                        'context': context,
                        'qas': qas
                    }]
                }
                
                unanswerable += 1
                data['data'].append(d)
    
            continue
        
        cxt_len = max(len(answer) + max_context_len // 2, max_context_len)
        
        if len(context) > cxt_len:
            start = max(0, finds[0] - random.randrange(0, cxt_len - len(answer)))
            oc = context
            context = context[start: start + cxt_len]
            
            finds = find_all(context, answer)
            
            assert len(finds) > 0
            
        qas = [{
            'question': question,
            'id': row.id + '_' + row.docid,
            'answers': [{
                'text': answer,
                'answer_start': f
            } for f in finds],
            'is_impossible': False,
        }]
        
        answerable += 1
        
#         for srow in ds.sample(5).itertuples():
#             sanswer = re.sub(r'\s+', ' ', srow.answer).strip()
            
#             if sanswer not in context:
#                 qas.append({
#                     'question': srow.question,
#                     'id': srow.id + '_' + row.docid,
#                     'answers': [],
#                     'is_impossible': True,
#                 })
#                 unanswerable += 1
#                 break
#         else:
#             print('!')

        d = {
            'title': row.docid,
            'paragraphs': [{
                'context': context,
                'qas': qas
            }]
        }

        data['data'].append(d)
        
    print('answerable/unanswerable:', answerable, unanswerable)
        
    return data

In [39]:
train_cut = train_doc_N3_docid.iloc[:45000,:]
val_cut = train_doc_N3_docid.iloc[45000:,:]

In [41]:
train_cut.head(2)

,id,docid,question,answer,y,text
0,47a41a03966431739257ef215cdc1caa,015758c216923f89991ca61c67b29f70,工业和信息化部到哪家企业进行督导检查？,北京北铃专用汽车有限公司,1,工业和信息化部组织开展负压救护车重点生产企业督导检查 2020年2月4日，为做好新型冠状病毒...
1,47a41a03966431739257ef215cdc1caa,028b66062e28322ebf200b483ecc31ee,工业和信息化部到哪家企业进行督导检查？,北京北铃专用汽车有限公司,0,关于启动我市重污染天气橙色预警（Ⅱ级） 各县、区人民政府、城乡一体化示范区管委会，市重污染天...


In [46]:
from tqdm import tqdm
import re
import random
def find_all(a, b):
    locs = []
    p = 0
    
    while True:
        q = a[p:].find(b)
        if q == -1:
            break
        else:
            locs.append(p + q)
            p = p + q + len(b)
    
    return locs

train_cut_make = get_data(train_cut)

100%|█████████████████████████████████████████████████████████████████████████| 45000/45000 [00:04<00:00, 10714.76it/s]

answerable/unanswerable: 8927 24614


In [47]:
valid_cut_make = get_data(val_cut)

100%|███████████████████████████████████████████████████████████████████████████| 5000/5000 [00:00<00:00, 12051.21it/s]

answerable/unanswerable: 1078 2579


In [67]:
#生成临时验证数据
train_cut_tem = train_doc_N3_docid.iloc[:500,:]
train_cut_make_tem = get_data(train_cut_tem)
with open(r'C:\Users\qian_wang\Desktop\疫情\squad_train_tem.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(train_cut_make_tem, ensure_ascii=False))

100%|██████████████████████████████████████████████████████████████████████████████| 500/500 [00:00<00:00, 5390.73it/s]

answerable/unanswerable: 94 286


In [51]:
import json
with open(r'C:\Users\qian_wang\Desktop\疫情\squad_train.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(train_cut_make, ensure_ascii=False))
    
with open(r'C:\Users\qian_wang\Desktop\疫情\squad_eval.json', 'w', encoding='utf-8') as f:
    f.write(json.dumps(valid_cut_make, ensure_ascii=False))

In [69]:
#生成测试数据集保留得分
def test_sample_docidN(test,docid_index,N):
    id_ = []
    question_ = []
    docid_ = []
    scores = []
    for i in range(test.shape[0]):
        dic = topN_doc(test["question"][i],bm25Model,average_idf,stop_flag,N)
        docids = []
        score_dic = []
        
        #添加分数，分数归一化
        #sum_ = sum([i[1] for i in dic]) 
        for k,v in dic:
            docids.append(docid_index[k])
            score_dic.append(v)
            #score_dic.append(v/sum_) 
        id_.extend([test["id"][i]]*N)
        docid_.extend(docids)
        question_.extend([test["question"][i]]*N)
        scores.extend(score_dic)
    return pd.DataFrame({"id":id_,"question":question_,"docid":docid_,"bm25_score":scores})
#切分预测集样本
def cut_test_docment(data,doc_span_length,reserve_length):
    id_ = []
    docid_ = []
    question_ = []
    texts_ = []
    bm25_scores = []
    count = 0
    for i in range(data.shape[0]):
        text_ = []
        a = data["text"].values[i]
        if len(a) > reserve_length:
            
            for j in range(0,len(a),doc_span_length):
                if j+reserve_length < len(a):
                    text_.append(a[j:(j+reserve_length)])
                else:
                    text_.append(a[j:])
            texts_.extend(text_)
            id_.extend([data["id"].values[i]]*len(text_))
            docid_.extend([data["docid"].values[i]]*len(text_))
            question_.extend([data["question"].values[i]]*len(text_))
            bm25_scores.extend([data["bm25_score"].values[i]]*len(text_))
            count+=len(text_)
       
        else:
            count+=1
            id_.append(data["id"].values[i])
            docid_.append(data["docid"].values[i])
            question_.append(data["question"].values[i])
            texts_.append(data["text"].values[i])
            bm25_scores.append(data["bm25_score"].values[i])
    print(len(id_),len(docid_),len(question_),len(texts_))
    return pd.DataFrame({"id":id_,"docid":docid_,"question":question_,"text":texts_,"bm25_score":bm25_scores}),count

#这里生成测试集，并将结果b25结果拼接,这里doc_pro先去重下
#doc_pro.drop_duplicates(subset = ["text"],keep='first',inplace = True)
test_sample_N =  test_sample_docidN(test,docid_index,30)
test_sample_N_docid = pd.merge(test_sample_N,doc_pro,how = "left",on = "docid")
test_sample_N_docid.shape

(49290, 5)

In [70]:
test_sample_N_docid.head()

,id,question,docid,bm25_score,text
0,0ce5c44a54d63a5b9b7c9657d1919fc7,天津市继续教育网在“抗疫知识专栏”中设有哪两类课程？,34830ac2ef5d314fbad41f7beed484f4,61.612972,市人社局关于在天津市专业技术人才继续教育网增设抗疫知识专栏的通知 市人社局关于在天津市专业技...
1,0ce5c44a54d63a5b9b7c9657d1919fc7,天津市继续教育网在“抗疫知识专栏”中设有哪两类课程？,5db3ed08679c33829a07ada724761177,27.306117,人社部疫情防控期间免费开放“技工教育网”并征集优质数字教学资源 近日，人力资源社会保障部发出...
2,0ce5c44a54d63a5b9b7c9657d1919fc7,天津市继续教育网在“抗疫知识专栏”中设有哪两类课程？,b1bab92d78c33780b45eff48fed3fc7a,18.096312,天津上线“艺术云课堂” 首批34位名家新秀以“艺”战“疫” 新华社天津2月17日电（记者 周...
3,0ce5c44a54d63a5b9b7c9657d1919fc7,天津市继续教育网在“抗疫知识专栏”中设有哪两类课程？,d9a671a44f9f30989037fc8a921ffe74,15.769269,关于推进天津市民营中小企业信用体系建设的意见 2014年8月22日 来源：中国人民银行天...
4,0ce5c44a54d63a5b9b7c9657d1919fc7,天津市继续教育网在“抗疫知识专栏”中设有哪两类课程？,a2f8532bad383367b5c4feb1fd72b75f,15.594158,市商务局 中国银行天津市分行关于对天津市“齐抗疫、稳发展、助企赢”专属金融服务有关内容进行调...


In [71]:
test_sample_N_docid.to_csv(r"C:\Users\qian_wang\Desktop\疫情\test_0512.csv",index = False)

In [83]:
import pickle
test_preds = pickle.load(open(r"C:\Users\qian_wang\Desktop\疫情\test_N_0514.p", 'rb'))

In [84]:
test_preds[0]

[{'score': 0.9930012609486454,
  'null_score': 1.246149258847387e-09,
  'start': 247,
  'end': 333,
  'answer': '程：“新冠肺炎防疫小课堂”主要发布有关新冠肺炎的科学知识，“防疫抗疫身心调节公益课堂”主要发布中国心理学会、清华大学等知名专家讲授的有关疫情防控期间心理健康调适方法和技巧。'},
 {'score': 0.6432894008745649,
  'null_score': 6.840837030033769e-09,
  'start': 247,
  'end': 333,
  'answer': '程：“新冠肺炎防疫小课堂”主要发布有关新冠肺炎的科学知识，“防疫抗疫身心调节公益课堂”主要发布中国心理学会、清华大学等知名专家讲授的有关疫情防控期间心理健康调适方法和技巧。'},
 {'score': 0.35066752567709614,
  'null_score': 6.840837030033769e-09,
  'start': 247,
  'end': 461,
  'answer': '程：“新冠肺炎防疫小课堂”主要发布有关新冠肺炎的科学知识，“防疫抗疫身心调节公益课堂”主要发布中国心理学会、清华大学等知名专家讲授的有关疫情防控期间心理健康调适方法和技巧。二、从即日起至疫情结束，本市专业技术人才以及广大公众，可直接登录继续教育网（网址：http://tjjxjy.chinahrt.com）首页，点击“抗疫知识专栏”；或在市人社局官网（网址：http://hrss.tj.gov.cn/）首页“最新公告”栏目，'},
 {'score': 0.0036248473080585097,
  'null_score': 1.246149258847387e-09,
  'start': 247,
  'end': 461,
  'answer': '程：“新冠肺炎防疫小课堂”主要发布有关新冠肺炎的科学知识，“防疫抗疫身心调节公益课堂”主要发布中国心理学会、清华大学等知名专家讲授的有关疫情防控期间心理健康调适方法和技巧。二、从即日起至疫情结束，本市专业技术人才以及广大公众，可直接登录继续教育网（网址：http://tjjxjy.china

In [90]:
test_sample_N_docid.rename(columns = {"bm25_score":"score"},inplace = True)

In [91]:
test_sample_N_docid.head()

,id,question,docid,score,text
0,0ce5c44a54d63a5b9b7c9657d1919fc7,天津市继续教育网在“抗疫知识专栏”中设有哪两类课程？,34830ac2ef5d314fbad41f7beed484f4,61.612972,市人社局关于在天津市专业技术人才继续教育网增设抗疫知识专栏的通知 市人社局关于在天津市专业技...
1,0ce5c44a54d63a5b9b7c9657d1919fc7,天津市继续教育网在“抗疫知识专栏”中设有哪两类课程？,5db3ed08679c33829a07ada724761177,27.306117,人社部疫情防控期间免费开放“技工教育网”并征集优质数字教学资源 近日，人力资源社会保障部发出...
2,0ce5c44a54d63a5b9b7c9657d1919fc7,天津市继续教育网在“抗疫知识专栏”中设有哪两类课程？,b1bab92d78c33780b45eff48fed3fc7a,18.096312,天津上线“艺术云课堂” 首批34位名家新秀以“艺”战“疫” 新华社天津2月17日电（记者 周...
3,0ce5c44a54d63a5b9b7c9657d1919fc7,天津市继续教育网在“抗疫知识专栏”中设有哪两类课程？,d9a671a44f9f30989037fc8a921ffe74,15.769269,关于推进天津市民营中小企业信用体系建设的意见 2014年8月22日 来源：中国人民银行天...
4,0ce5c44a54d63a5b9b7c9657d1919fc7,天津市继续教育网在“抗疫知识专栏”中设有哪两类课程？,a2f8532bad383367b5c4feb1fd72b75f,15.594158,市商务局 中国银行天津市分行关于对天津市“齐抗疫、稳发展、助企赢”专属金融服务有关内容进行调...


In [125]:
from rouge.rouge import Rouge
from difflib import SequenceMatcher
import copy
import re

import thulac
thu = thulac.thulac(seg_only=True, user_dict=r"C:\Users\qian_wang\Desktop\疫情\user_dict.txt")

import sys
sys.setrecursionlimit(10000)

rouge = Rouge()

def spacify(s):
    return re.sub(r'\s+', ' ', ' '.join(list(s)))

def get_result(model_preds, df_search, a, b, c, w1, w2, w3, w4):
    model_preds = copy.deepcopy(model_preds)
    
    max_answer_len = 2500
    answers = []
    for i, p in enumerate(model_preds):
        cands = []
        min_null_score = 10000
        for j, d in enumerate(p):
            if d['answer'] == '':
                min_null_score = min(min_null_score, d['score'])
            else:
                d['score1'] = c * d['score'] + (1 - c) * (1 - d['null_score'])
            
                
#                 if len(d['answer']) < 2 or len(d['answer']) > max_answer_len:
#                     d['score1'] = max(0, d['score1'] - 0.1)
                cands.append(d)
        
        cands = sorted(cands, key=lambda x:x['score1'], reverse=True)
        #print(cands)
        s1 = cands[0]['score1'] if len(cands) > 0 else 0
        s2 = cands[1]['score1'] if len(cands) > 1 else 0
        s3 = cands[2]['score1'] if len(cands) > 2 else 0
        s4 = cands[3]['score1'] if len(cands) > 3 else 0
        
        score_context = w1 * s1 + w2 * s2 + w3 * s3 + w4 * s4

        best_answer = cands[0]['answer'].strip()

        if min_null_score > 1:
            min_null_score = 0
            
        if len(best_answer) < 3 or len(best_answer) > max_answer_len:
            score_context = score_context - 0.5
            
        answers.append((best_answer, min_null_score, score_context))
        
    df_preds = pd.DataFrame(answers, columns=['pred', 'null_prob', 'prob'])
    #print(df_preds)
    df_search_pred = pd.concat([df_search.reset_index(drop=True), df_preds], axis=1)
#         df_search_pred = df_search_pred[df_preds['null_prob'] < thres]

    df_search_pred['score_all'] = df_search_pred['score'] * (1 - a - b) / 100 + df_search_pred['prob'] * a + (1 - df_search_pred['null_prob']) * b

    df_pred_select = []
    for k, _df in df_search_pred.groupby('id'):
        df_pred_select.append(_df.sort_values('score_all', ascending=False).iloc[0])

    df_pred_select = pd.DataFrame(df_pred_select)
    #print(df_pred_select.head())
    answer_pred_refine = []
    for row in df_pred_select.itertuples():
        matches = [m for m in SequenceMatcher(a=row.text, b=row.pred).get_matching_blocks() if m.size > 0]
        print(matches)
        refine = row.text[matches[0].a: matches[-1].a + matches[-1].size]
        
        old_refine = copy.copy(refine)
        
        refine = refine.strip('，').strip(':').strip('：').strip('（').strip('；').strip('、').strip()
        
        if re.sub(r'\s+', '', refine) != re.sub(r'\s+', '', row.pred):
            refine = row.pred
        
        if len(refine) > 5 and len(re.findall('[。，：]', refine[:4])) > 0:
            for f in re.finditer('[。，：、]', refine[:4]):
                refine = refine[f.span()[0] + 1:]
                break
                
            print('----截取开头标点----')
        
        
        if re.match(r'^[的].*', refine):
            refine = refine[1:]
            print('----去掉开头停用词----')
            
        if matches[0].a != 0:
            tmp = row.text[matches[0].a - 1: matches[-1].a + matches[-1].size]
            tw = [w[0] for w in thu.cut(tmp)]
            rw = [w[0] for w in thu.cut(refine)]
            
            if len(rw) > 0 and rw[0] != tw[0] and rw[0] in tw[0]: # 少了一个字
                print('----去掉开头不完整词语----')
                refine = tmp[len(tw[0]):]
                
#         if matches[-1].a + matches[-1].size < len(row.text):
#             tmp = row.text[matches[0].a: matches[-1].a + matches[-1].size + 1]
#             tw = [w[0] for w in thu.fast_cut(tmp)]
#             rw = [w[0] for w in thu.fast_cut(refine)]
#             if len(rw) > 0 and rw[-1] != tw[-1] and rw[-1] in tw[-1]:
#                 print('----去掉结尾不完整词语----')
#                 refine = tmp[: len(tmp) - len(tw[-1])]
                
        refine = refine.strip('，').strip(':').strip('：').strip('（').strip('；').strip('、').strip()
                
        if old_refine != refine:
            print('old:', old_refine)
            print('new:', refine)
            
        answer_pred_refine.append(refine)

    df_pred_select['answer_pred_refine'] = answer_pred_refine

    return df_pred_select
df_pred_select = get_result(test_preds, test_sample_N_docid, 0.3, 0, 0, 1, 0, 0, 0)

Model loaded succeed
[Match(a=0, b=0, size=21), Match(a=22, b=21, size=15), Match(a=38, b=36, size=114), Match(a=153, b=150, size=68), Match(a=222, b=218, size=159)]
[Match(a=482, b=0, size=89)]
----截取开头标点----
old: 点：一是通过群众喜闻乐见的各种形式宣传科学防控；二是通过宣传好人好事、提供各类人性化服务，传递社区温度，取得群众支持；三是全面掌握社情民意，妥善处置苗头性问题，及时引导正确舆论。
new: 一是通过群众喜闻乐见的各种形式宣传科学防控；二是通过宣传好人好事、提供各类人性化服务，传递社区温度，取得群众支持；三是全面掌握社情民意，妥善处置苗头性问题，及时引导正确舆论。
[Match(a=271, b=0, size=21)]
[Match(a=28, b=0, size=46)]
[Match(a=137, b=0, size=26)]
----去掉开头不完整词语----
old: 台（https://www.zjmuex.com）”
new: https://www.zjmuex.com）”
[Match(a=259, b=0, size=36)]
[Match(a=201, b=0, size=8)]
----去掉开头不完整词语----
old: 计2.36亿元；
new: 2.36亿元
[Match(a=616, b=0, size=75)]
[Match(a=2040, b=0, size=15)]
[Match(a=214, b=0, size=84)]
[Match(a=23, b=0, size=4)]
[Match(a=494, b=0, size=8)]
[Match(a=308, b=0, size=50)]
[Match(a=377, b=0, size=119)]
[Match(a=164, b=0, size=137)]
[Match(a=124, b=0, size=21)]
[Match(a=945, b=0, size=15)]
[Match(a=586, b=0, size=5)]
[Match(a=269, b=0

old: 话：18600636475、13520558904
new: 18600636475、13520558904
[Match(a=395, b=0, size=108), Match(a=505, b=108, size=113)]
[Match(a=476, b=0, size=68)]
[Match(a=439, b=0, size=8)]
[Match(a=1043, b=0, size=95)]
[Match(a=996, b=0, size=96)]
[Match(a=87, b=0, size=18)]
old: 新疆开通了防疫物资专用通道566条，
new: 新疆开通了防疫物资专用通道566条
[Match(a=0, b=0, size=16)]
[Match(a=325, b=0, size=49)]
[Match(a=85, b=0, size=48), Match(a=134, b=48, size=157), Match(a=292, b=205, size=89), Match(a=382, b=294, size=79)]
[Match(a=192, b=0, size=23)]
old: 因疫情影响无法按时提交续展申请的，可延期提交；
new: 因疫情影响无法按时提交续展申请的，可延期提交
[Match(a=169, b=0, size=8)]
[Match(a=146, b=0, size=41)]
[Match(a=89, b=0, size=31)]
[Match(a=444, b=0, size=16)]
----去掉开头不完整词语----
old: 门《关于禁止野生动物交易的公告》
new: 《关于禁止野生动物交易的公告》
[Match(a=0, b=0, size=3)]
[Match(a=154, b=0, size=8)]
old: 免费向社会开放，
new: 免费向社会开放
[Match(a=319, b=0, size=32)]
old: 必须取得并随身携带居住地所在乡镇（街道）以上单位出具的健康证明；
new: 必须取得并随身携带居住地所在乡镇（街道）以上单位出具的健康证明
[Match(a=21, b=0, size=11)]
----去掉开头不完整词语----
old: 在线报送和远程监测调度
new: 

[Match(a=214, b=0, size=58)]
old: 对象范围从确诊借款人扩大到因受疫情影响还款出现困难的确诊、疑似、无法排除、密切接触的集中收治集中隔离的“四类人员”，
new: 对象范围从确诊借款人扩大到因受疫情影响还款出现困难的确诊、疑似、无法排除、密切接触的集中收治集中隔离的“四类人员”
[Match(a=90, b=0, size=103)]
[Match(a=534, b=0, size=124)]
[Match(a=639, b=0, size=16)]
[Match(a=669, b=0, size=74)]
[Match(a=67, b=0, size=6)]
----去掉开头不完整词语----
old: 记2月10日
new: 2月10日
[Match(a=804, b=0, size=39)]
old: 登录广西壮族自治区&ldquo;数字人社&rdquo;网上服务大厅进行申请。（
new: 登录广西壮族自治区&ldquo;数字人社&rdquo;网上服务大厅进行申请。
[Match(a=171, b=0, size=19), Match(a=195, b=19, size=12)]
[Match(a=67, b=0, size=10)]
old: 自1月28日6时起，
new: 自1月28日6时起
[Match(a=251, b=0, size=6)]
----去掉开头不完整词语----
old: 助70多万人
new: 70多万人
[Match(a=298, b=0, size=22)]
[Match(a=69, b=0, size=109)]
[Match(a=335, b=0, size=34)]
[Match(a=604, b=0, size=47)]
[Match(a=879, b=0, size=10)]
old: 制定“一楼一方案”，
new: 制定“一楼一方案”
[Match(a=407, b=0, size=17)]
[Match(a=257, b=0, size=118), Match(a=376, b=118, size=82)]
[Match(a=390, b=0, size=107)]
[Match(a=322, b=0, size=51), Match(a=374, b=51, size=53)]
ol

[Match(a=237, b=0, size=26)]
[Match(a=107, b=0, size=66), Match(a=178, b=66, size=35), Match(a=218, b=101, size=24), Match(a=247, b=125, size=38), Match(a=290, b=163, size=20)]
[Match(a=0, b=0, size=3)]
old: 浙江：
new: 浙江
[Match(a=445, b=0, size=68)]
[Match(a=386, b=0, size=134), Match(a=521, b=134, size=241)]
[Match(a=427, b=0, size=26)]
[Match(a=128, b=0, size=41)]
[Match(a=737, b=0, size=106)]
old: 市人社部门通过系统数据筛查出符合申领稳岗返还待遇的企业，审核发放後通过发送短信、推送网上服务大厅的方式通知企业确认申领。企业也可以自行登录广西壮族自治区&ldquo;数字人社&rdquo;网上服务大厅进行申请。（
new: 市人社部门通过系统数据筛查出符合申领稳岗返还待遇的企业，审核发放後通过发送短信、推送网上服务大厅的方式通知企业确认申领。企业也可以自行登录广西壮族自治区&ldquo;数字人社&rdquo;网上服务大厅进行申请。
[Match(a=164, b=0, size=26)]
old: 市民基本生活、公共事务办理、公共场所设施等各个方面，
new: 市民基本生活、公共事务办理、公共场所设施等各个方面
[Match(a=283, b=0, size=14)]
old: 延长至2020年3月31日，
new: 延长至2020年3月31日
[Match(a=2071, b=0, size=22)]
[Match(a=342, b=0, size=36)]
[Match(a=577, b=0, size=10)]
[Match(a=248, b=0, size=239)]
[Match(a=273, b=0, size=12)]
----去掉开头不完整词语----
old: 台——“战疫金盾”系统，
new: ——“战疫金盾”系统
[Match(a=195, b=0, s

old: 习近平总书记重要指示精神，全面落实党中央、国务院关于当前疫情防控工作的决策部署和税务总局及四川省委省政府工作要求，充分释放税费政策效应，
new: 习近平总书记重要指示精神，全面落实党中央、国务院关于当前疫情防控工作的决策部署和税务总局及四川省委省政府工作要求，充分释放税费政策效应
[Match(a=341, b=0, size=151)]
[Match(a=477, b=0, size=15)]
----去掉开头不完整词语----
old: 注“海陵审批服务”微信公众号，
new: “海陵审批服务”微信公众号
[Match(a=460, b=0, size=35)]
[Match(a=700, b=0, size=40)]
[Match(a=485, b=0, size=25)]
----去掉开头不完整词语----
old: 线下或利用自有网站、第三方培训平台线上职业培训的，
new: 下或利用自有网站、第三方培训平台线上职业培训的
[Match(a=227, b=0, size=9)]
----去掉开头不完整词语----
old: 付1.29亿元资金
new: 1.29亿元资金
[Match(a=782, b=0, size=19)]
old: 允许延期至疫情解除后三个月内补办补缴；
new: 允许延期至疫情解除后三个月内补办补缴
[Match(a=1500, b=0, size=54)]
old: 对一线医务人员及其家属口罩等防护用品的配给，优先保障辖区内定点收治医院及援助湖北医疗队队员防护用品的供应。（
new: 对一线医务人员及其家属口罩等防护用品的配给，优先保障辖区内定点收治医院及援助湖北医疗队队员防护用品的供应。
[Match(a=0, b=0, size=3)]
[Match(a=163, b=0, size=10)]
----去掉开头不完整词语----
old: 取在线填写问卷方式。
new: 在线填写问卷方式。
[Match(a=210, b=0, size=34)]
[Match(a=43, b=0, size=4)]
----去掉开头不完整词语----
old: 还50%
new: 50%
[Match(a=665, b=0, size=23)]
[Match(a=39, b=0, size=67)]
[Match(a=0, b=0

...

[Match(a=125, b=0, size=1), Match(a=127, b=1, size=21), Match(a=149, b=22, size=29), Match(a=179, b=51, size=20), Match(a=200, b=71, size=22), Match(a=223, b=93, size=19)]
[Match(a=154, b=0, size=42)]
[Match(a=107, b=0, size=34)]
old: 重庆市电子税务局、自然人电子税务局、“渝快办”、微信公众号、支付宝、
new: 重庆市电子税务局、自然人电子税务局、“渝快办”、微信公众号、支付宝
[Match(a=7, b=0, size=3)]
[Match(a=790, b=0, size=61)]
[Match(a=390, b=0, size=147)]
[Match(a=833, b=0, size=42)]
[Match(a=1163, b=0, size=39)]
[Match(a=79, b=0, size=3)]
[Match(a=119, b=0, size=12)]
----去掉开头不完整词语----
old: 区）一律实行封闭式管理，
new: ）一律实行封闭式管理
[Match(a=282, b=0, size=13)]
[Match(a=147, b=0, size=42)]
[Match(a=129, b=0, size=84)]
old: （1）微信办理：通过加入宿州市应急物资运输通行证办理群申报办理。 （2）电子邮箱办理：通过市疫情防控指挥部交通管控组电子邮箱（szyjtxz@163.com）申报办理。
new: 1）微信办理：通过加入宿州市应急物资运输通行证办理群申报办理。 （2）电子邮箱办理：通过市疫情防控指挥部交通管控组电子邮箱（szyjtxz@163.com）申报办理。
[Match(a=759, b=0, size=25)]
old: 卫生员数量不少于2人，且不少于全部工人数量的2%，
new: 卫生员数量不少于2人，且不少于全部工人数量的2%
[Match(a=107, b=0, size=11)]
[Match(a=1754, b=0, size=82), Match(a=1838, b=82, size

[Match(a=91, b=0, size=110)]
[Match(a=1066, b=0, size=139)]
[Match(a=128, b=0, size=29)]
[Match(a=103, b=0, size=58)]
[Match(a=211, b=0, size=39)]
[Match(a=337, b=0, size=62)]
[Match(a=372, b=0, size=110)]
[Match(a=159, b=0, size=6)]
[Match(a=459, b=0, size=143)]
[Match(a=331, b=0, size=37)]
old: 网上查询、网上预约、网上办理，对确需现场办理的事项，电话咨询、预约办理。（
new: 网上查询、网上预约、网上办理，对确需现场办理的事项，电话咨询、预约办理。
[Match(a=339, b=0, size=75)]
[Match(a=1248, b=0, size=107), Match(a=1356, b=107, size=14), Match(a=1371, b=121, size=107)]
[Match(a=232, b=0, size=28)]
[Match(a=184, b=0, size=63)]
[Match(a=468, b=0, size=111)]
[Match(a=4452, b=0, size=253)]
[Match(a=405, b=0, size=7)]
old: 达61.9%，
new: 达61.9%
[Match(a=546, b=0, size=119)]
[Match(a=384, b=0, size=101)]
[Match(a=266, b=0, size=184)]
[Match(a=120, b=0, size=9)]
[Match(a=176, b=0, size=6)]
[Match(a=197, b=0, size=29)]
[Match(a=264, b=0, size=121)]
[Match(a=614, b=0, size=22), Match(a=637, b=22, size=232)]
[Match(a=475, b=0, size=11)]
[Match(a=222, b=0, size=62)]
old: 

old: 自然人涉税服务网、厦门税务微服务等“非接触式”途径，减少现场办税次数，最大程度降低交叉感染的风险。相关事项提示如下： 一、延长2月份申报期限 根据疫情防控需要，按照国家税务总局的要求，全国范围内2月份纳税申报期限延长至2月24日。对受疫情影响办理申报有困难的纳税人可在此基础上申请延长期限。 二、常见涉税（费）业务办理 （一）厦门市电子税务局 提供“我要办税”“我要查询”“互动中心”“公众服务”等业务办理功能，常见涉税（费）业务均可通过电子税务局办理。详情请登录国家税务总局厦门市电子税务局浏览。网址：https://etax.xiamen.chinatax.gov.cn （二）厦门税务掌上办税厅 打开微信或支付宝，搜索“厦门税务掌上办税厅”小程序，并点击进入，提供实名信息采集、发票代开、发票申领、发票核定调整、一般纳税人登记、新办纳税人套餐式服务申请、车辆购置税信息查询等业务办理功能。 （三）自然人电子税务局 提供个人所得税相关申报业务、企业扣缴及代理申报。详情请登录自然人电子税务局浏览。网址：https://etax.chinatax.gov.cn （四）自然人涉税服务网 提供自然人增值税普通发票代开、增值税专用发票代开、自然人房屋出租发票代开办理。详情请登录自然人涉税服务网浏览。网址：https://zrr.xiamen.chinatax.gov.cn/portals/web/login （五）厦门税务微服务 在微信搜索框中输入“厦门税务微服务”关注并进入“厦门税务微服务”微信公众号，提供办税预约、社保服务、退税查询、涉税费文书查询、发票查询、完税证明查询、通知公告、优惠政策等业务功能。 （六）涉税咨询 1.涉税咨询业务： (1)政策咨询、投诉举报、意见建议等可在工作日工作时间通过拨打纳税服务热线12366办理。 (2)除了电话咨询，您可以关注“厦门税务微服务”公众号，进行微信在线咨询；或者登录厦门税务网站，点击“我要咨询”模块进行智能咨询、在线咨询或留言咨询。 2.我局将开通“防疫情12366绿色通道”。对影响企业生产经营，且不能远程办理的复杂事项，您可以拨打12366热线，由热线登记後转具体办理部门进行预沟通，提速现场办税，最大限度减少人员在办税服务大厅的停留时间。 （七）办税服务厅 1．根据国务院办公厅关于延长2020年春节假期的通知，全市办税服务

[Match(a=105, b=0, size=80), Match(a=186, b=80, size=94)]
[Match(a=311, b=0, size=6)]
[Match(a=88, b=0, size=20)]
old: 通过政府购买服务、企业及群众自愿等形式，
new: 通过政府购买服务、企业及群众自愿等形式
[Match(a=484, b=0, size=4)]
----去掉开头停用词----
old: 的50%
new: 50%
[Match(a=2931, b=0, size=4)]
[Match(a=16, b=0, size=21)]
----去掉开头不完整词语----
old: 发《施工现场新型冠状病毒突发疫情应急预案》
new: 《施工现场新型冠状病毒突发疫情应急预案》
[Match(a=320, b=0, size=69)]
[Match(a=633, b=0, size=47)]
[Match(a=893, b=0, size=84)]
old: 加大对生活必需品供应企业协调和服务保障力度，对保证市民日常生活需要的大型超市、药店、菜场等重点疫情防疫保供服务业企业，按照疫情期间经营用电费、水费等情况给予适当补助。（
new: 加大对生活必需品供应企业协调和服务保障力度，对保证市民日常生活需要的大型超市、药店、菜场等重点疫情防疫保供服务业企业，按照疫情期间经营用电费、水费等情况给予适当补助。
[Match(a=209, b=0, size=43)]
[Match(a=72, b=0, size=8)]
old: 授信350亿元，
new: 授信350亿元
[Match(a=857, b=0, size=74)]
[Match(a=148, b=0, size=9)]
----去掉开头不完整词语----
old: 治“三集中”方案，
new: “三集中”方案
[Match(a=254, b=0, size=26)]
[Match(a=425, b=0, size=11)]
----去掉开头不完整词语----
old: 写《旅客信息登记表》。
new: 《旅客信息登记表》。
[Match(a=935, b=0, size=20)]
[Match(a=1366, b=0, size=5)]
old: 卿小云”，
new: 卿小云”
[Mat

[Match(a=450, b=0, size=53)]
----截取开头标点----
old: 话：0933-8231733/8223203、13085991878（捐款）18793341615（捐物）
new: 0933-8231733/8223203、13085991878（捐款）18793341615（捐物）
[Match(a=210, b=0, size=47)]
[Match(a=109, b=0, size=11)]
[Match(a=171, b=0, size=187)]
[Match(a=10, b=0, size=8)]
----去掉开头不完整词语----
old: 均8.8万辆/次
new: 8.8万辆/次
[Match(a=343, b=0, size=206)]
[Match(a=919, b=0, size=211), Match(a=1133, b=211, size=54)]
[Match(a=153, b=0, size=68)]
[Match(a=749, b=0, size=84)]
[Match(a=203, b=0, size=52)]
old: 了“新型冠状病毒肺炎确诊患者同行程查询”“全省发热门诊地图”“疫情防控期间出入境信息查询”等多项功能。（
new: 了“新型冠状病毒肺炎确诊患者同行程查询”“全省发热门诊地图”“疫情防控期间出入境信息查询”等多项功能。
[Match(a=488, b=0, size=63)]
[Match(a=738, b=0, size=27)]
[Match(a=696, b=0, size=139), Match(a=839, b=139, size=69)]
old: 在辽中区、新民市、法库县、康平县建设10个粪污区域收集中心、3个区域粪污处理中心（有机肥厂）。推进10个现有畜禽规模养殖场粪便污水治理设施改造升级，加快市级和新民市病死动物无害化处理中心建设，2020年年底前全部建成并投入运行。（责任部门：市农业农村局及有关区、县（市）政府）     （四）规范禁养区划定和管理。根据国家有关法律和规定科学划定禁养区，自查和调整超范围划定的禁养区，并上报自查结果和调整後的禁养区划定情况。（
new: 在辽中区、新民市、法库县、康平县建设10个粪污区域收集中心、3个区域粪污处理中心（有机肥厂）

In [122]:
thu.cut("我们一起去长城")

[['我们', ''], ['一起', ''], ['去', ''], ['长城', '']]

In [127]:
df_pred_select.shape

(1643, 10)

In [128]:
df_rst = df_pred_select[['id', 'docid', 'answer_pred_refine']]
df_rst.columns = ['id', 'docid', 'answer']
df_rst

,id,docid,answer
30000,000237f617a835f89a525db97b3bdbff,5843627390ae3eba8d81697ce95a145d,陕西新改造1.2万公里农村公路助力脱贫攻坚 新华社西安12月24日电（记者 薛天）记者24日...
14730,0008bd92faaf330796e28e7bbef388a5,7484916ce1363d7baad2b41728be8a6b,一是通过群众喜闻乐见的各种形式宣传科学防控；二是通过宣传好人好事、提供各类人性化服务，传递社...
27360,0086edef25b33cc48beace3b3b887a52,b9f92ea490ac34c382294a67fc43024a,将《苏州市旅游景区景点、体育场馆复工指南》
41520,0094268b34103f53893010e457d2fcfe,d91e7aa2bcd6379e958d35b63c6bb593,对在疫情防控期间不能按期办理驾驶证审验换证、机动车检验等业务的，可以延期到疫情结束以后办理。
870,00ffb393d0063cb39bfc7e63c46cf65a,13a32c4a820a30f89540d3ed15da2df6,https://www.zjmuex.com）”
47850,0160549e64153e3e86d2953dd4ab63cc,e54ea064692c322a88dca26d36410f3d,不加收税款滞纳金、不给予行政处罚、不调整纳税信用评价、不认定为非正常户。
35430,01938b5e30fc3481b660ab9209cdb240,b384f427eb72310db60745c08bb12686,2.36亿元
47250,019e56e6c3d039e2aff57f0f621a01e3,e806a23e6a233ffc86c6f37ecf17382a,养老机构应当及时与老年人和家属沟通商量，达成一致後，通过机构内医务人员处置、电话求助医疗机构...
3150,01f05ca6015c33b796bb6e1ebc15e199,076b9477df8d3f4495ac50ca68703d49,民声通道、局长信箱、网上信访等
46560,020e11ee430b345583457f5629f96577,f4229f46579e368c93aeb06b8440b68f,所有在川居住14天以上且接受当地村（社区）管理的有外出务工需要的人员，如需跨市（州）以上流动...


In [130]:
df_rst.to_csv(r"C:\Users\qian_wang\Desktop\疫情\test_new_0515.csv", sep='\t', index=None)

In [ ]:
#训练
python gf_run_squad.py \
--model_type bert \
--model_name_or_path /home/wq/bert_wwm/pytorch_model.bin \
--config_name /home/wq/bert_wwm/bert_config.json \
--tokenizer_name /home/wq/bert_wwm/vocab.txt \
--do_train \
--do_eval \
--do_lower_case \
--train_file /home/wq/squad_train.json \
--predict_file /home/wq/squad_eval.json \
--per_gpu_train_batch_size 6 \
--learning_rate 3e-5 \
--num_train_epochs 4 \
--max_seq_length 512 \
--max_query_length 80 \
--max_answer_length 400 \
--warmup_steps 2500 \
--doc_stride 128 \
--version_2_with_negative \
--output_dir /home/wq/torch0512